## The goal of collecting this dataset:

1- Identify Key Stress Factors: To determine the main contributors to student stress, such as academic workload, social life, and financial concerns.

2- Analyze Correlations: To explore how different stressors interact, like the impact of financial pressures on academic stress or the role of social support in alleviating stress.

3- Predict Impact: To assess how stress factors influence academic performance and well-being, identifying critical points where stress begins to negatively affect students.

4- Develop Interventions: To design targeted solutions like time management workshops or financial aid based on identified stress factors.

5- Enhance Learning Environments: To create healthier, more supportive educational environments by reducing stress through data-driven strategies.


## General information about the dataset:

Number of attributes: 21

Attribute types: All columns are integer types (int64)

Number of objects: 1100

Class/Label: stress_level

### The source of the dataset:
https://www.kaggle.com/datasets/rxnach/student-stress-factors-a-comprehensive-analysis